# OTUS | $p p > Z > e^+ e^-$

In [ ]:
# Based on the streamlined training procedure in experiments/ppzee/alternateSeedTests/ppzee-streamlined-seed=1.ipynb
# at 7e31a97, but with various settings of the anchor loss weight ('nu') in the initial training stage (the second
# stage of training turns off the anchor loss) while keeping lamb fixed at 1, in order to test out the effect of
# the anchor loss weight hyperparameter on the results. 

This notebooks applies OTUS to our first test case: $Z$ boson decaying into an electron ($e^-$) positron ($e^+$).

Our physical latent-space is the $e^+$, $e^-$ 4-momentum information produced by the program MadGraph and our data-space data is the $e^+$, $e^-$ 4-momentum information produced by the program Delphes.

We arrange this information into 8 dimensional vectors
- Latent space (z): [$p^{\mu}_{e^-}$,$p^{\mu}_{e^+}$]
- Data space (x):   [$p^{\mu}_{e^-}$,$p^{\mu}_{e^+}$]

where $p^{\mu}=[p_x, p_y, p_z, E]$ is the 4-momentum of the given particle.

###### Additional Losses and Constraints:
We impose the following additional losses and constraints in this problem.

First, we impose a constraint on the learned mappings via "anchor losses". This constrains the direction of the electron's 3-momenta when transforming from x-space to z-space and vice versa.

Second, we explicitly enforce the Minkowski metric in the output of the networks. Namely, the networks predict the 3-momenta ($\vec{p}$) of the particles. Energy information is then restored using the Minkowski metric: $E^2 = |\vec{p}|^2 + m^2$.

See the paper for more details: https://arxiv.org/abs/2101.08944.

# Load Required Libraries

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import numpy as np
import os

root_dir = '../../../'

#-- Add utilityFunctions/ to easily use utility .py files --#
import sys
sys.path.append(os.path.join(root_dir, "utilityFunctions/"))

#-- Determine if using GPU or CPU --#
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'  # Set to '-1' to disable GPU
from configs import device, data_dims

print('Using device:', device)

Using device: cuda


# Meta Parameters

In [3]:
data_directory    = os.path.join(root_dir, "data/")
dataset_name      = 'ppzee'

#-- Set random seeds --#
seed = 1
torch.manual_seed(seed)
np.random.seed(seed)

#-- Set data type --#
from configs import float_type
print('Using data type: ', float_type)

Using data type:  float32


# Load Data

In [4]:
from func_utils import get_dataset, standardize
from torch.utils.data import DataLoader


#-- Get training and validation dataset --#
dataset = get_dataset(dataset_name, data_dir=data_directory) 
z_data, x_data = dataset['z_data'], dataset['x_data']

MET = False # Exclude Missing Transverse Energy (MET) from x-space data
if MET == False:
    x_data = x_data[:, :-4]
print("Data total shapes: ",z_data.shape, x_data.shape)

x_dim = int(x_data.shape[1])
z_dim = int(z_data.shape[1])

#-- Split into training and validation sets --#
train_size = 291699
val_size = 40000  # Validation set used to evaluate/tune models

x_train = x_data[:train_size, :]
x_val = x_data[train_size:train_size+val_size, :]

z_train = z_data[:train_size, :]
z_val = z_data[train_size:train_size+val_size, :]

#-- Convert data to proper type --#
x_train, x_val, z_train, z_val = list(map(lambda x: x.astype(float_type), [x_train, x_val, z_train, z_val]))

#-- Obtain mean and std information --#
# This is needed to standardize/unstandardize data
x_train_mean, x_train_std = np.mean(x_train, axis=0), np.std(x_train, axis=0) 
z_train_mean, z_train_std = np.mean(z_train, axis=0), np.std(z_train, axis=0)

#-- Set evaluation parameters --#
eval_batch_size = 20000  # Always use high batch size on validation set to accurately assess performance
eval_loaders = DataLoader(dataset=x_val, batch_size=eval_batch_size, shuffle=True), \
               DataLoader(dataset=z_val, batch_size=eval_batch_size, shuffle=True)

print("z_train shape, x_train shape: ", z_train.shape, x_train.shape)
print("z_val   shape, x_val   shape: ", z_val.shape, x_val.shape)


Data total shapes:  (331699, 8) (331699, 8)
z_train shape, x_train shape:  (291699, 8) (291699, 8)
z_val   shape, x_val   shape:  (40000, 8) (40000, 8)


### Define target invariant masses (for both training and validation data)

Invariant mass relation: $m^2 = E^2 - |\vec{p}|^2$ 

In [5]:
#-- Look at m2 values empirically --#
print('X space')
tmp = x_data
for i in range(int(tmp.shape[1] / 4)):
    p = tmp[:, 4*i : 4*i+3]  # 3-momentum, (px, py, pz)
    E = tmp[:, 4*i+3]        # Energy
    m2 = (E**2 - (p ** 2).sum(axis=-1))
    print('particle %d mass^2 = %g +- %g' %(i, m2.mean(), m2.std()))

print('Z space')
tmp = z_data
for i in range(int(tmp.shape[1] / 4)):
    p = tmp[:, 4*i : 4*i+3]  # 3-momentum, (px, py, pz)
    E = tmp[:, 4*i+3]        # Energy
    m2 = (E**2 - (p ** 2).sum(axis=-1))
    print('particle %d mass^2 = %g +- %g' %(i, m2.mean(), m2.std()))

X space
particle 0 mass^2 = 2.01129e-15 +- 3.62862e-12
particle 1 mass^2 = -1.17544e-14 +- 3.50494e-12
Z space
particle 0 mass^2 = -1.34586e-09 +- 6.17582e-07
particle 1 mass^2 = -1.48398e-10 +- 5.71702e-07


Note that the fact that these values are slightly negative is a purely numerical effect. Thus we instead define them to the value they should have, namely 0. (massless).

In [6]:
#-- Define target invariant masses --#
x_inv_masses = np.zeros(2)
z_inv_masses = np.zeros(2)

# Train

## Import Training Specific Libraries and Functions

In [7]:
import torch
from torch import optim
from ppzee_utils import train_and_val

## Define Meta Network Parameters

In [8]:
cond_noise = True  # Whether to use conditional Gaussian (instead of standard normal) for noise in enc/dec
if cond_noise:
    from models import CondNoiseAutoencoder
    Autoencoder = CondNoiseAutoencoder  # Define alias 
else:
    from models import Autoencoder

## Define Model and Hyperparameters

###### Latent loss function: 
Finite sample approximation of Sliced Wasserstein Distance (SWD) between $p(z)$ and $p_E(z) = \int_x p(x) p_E(z|x)$
- $L_{latent}(Z, \tilde{Z}) = \frac{1}{L * M} \sum_{l=1}^{L} \sum_{m=1}^{M} c((\theta_l \cdot z_m)_{sorted}, (\theta_l \cdot \tilde{z}_m)_{sorted})$ 

where $c(\cdot, \cdot) = |\cdot - \cdot|^2$

###### Data loss function: 
- $L_{data}(X, \tilde{X}) = \frac{1}{M} \sum_{m=1}^M c(x_m,  \tilde{x}_m)$

where $c(\cdot, \cdot) = |\cdot - \cdot|^2$

###### Additional loss functions: 
Encoder and decoder anchor losses
- $L_{A}(X, \tilde{Z}) = \frac{1}{M} \sum_{m=1}^M c_A(x_m, \tilde{z}_m)$
- $L_{A}(Z, \tilde{X}') = \frac{1}{M} \sum_{m=1}^M c_A(z_m, \tilde{x}'_m)$

see the paper for additional details about $c_A$.

###### Full loss function:
- $L_{tot} = \beta L_{data}(X, \tilde{X}) + \lambda L_{latent}(Z, \tilde{Z}) + \nu_e L_{A}(X, \tilde{Z}) + \nu_d L_{A}(Z, \tilde{X}')$

###### Core Hyperparameters
The hyperparameter definitions are as follows:
- num_hidden_layers:    The number of hidden layers in both the encoder and decoder networks
- dim_per_hidden_layer: The dimensions per hidden layer in both the encoder and decoder networks
- lr: The learning rate of the networks
- lamb: The $\lambda$ coefficient in front of the latent loss term
- num_slices: Number of random projections used for computing SWD
- epochs: The number of epochs used during training

Hyperparameters for other losses that were tried, but use during main training is currently discouraged:
- tau: Coefficient in front of the alternate data-space loss ("alt_x_loss"), which is the SWD between $p(x)$ and $p_D(x):=\int_z p(z) p_D(x|z)$
- rho: Coefficient in front of an additional decoder constraint loss (based on soft-penalty approach to learning hard thresholds/ttbar_constraints)

###### Joint Training Hyperparameters

- beta: Coefficient in front of data loss, $L_{data}$     
- nu_e: Coefficient in front of the encoder "anchor loss" 
- nu_d: Coefficient in front of the decoder "anchor loss" 

In [9]:
config = {
    'num_hidden_layers': 1,
    'dim_per_hidden_layer': 128,
    'lr': 0.001,
    'beta': 1.,  # weight of the data reconstruction loss
    'lamb': 1.,  # weight of the latent space matching loss
    'tau': 0,  
    'rho': 0,  
}



num_hidden_layers, dim_per_hidden_layer = config['num_hidden_layers'], config['dim_per_hidden_layer']
hidden_layer_dims = num_hidden_layers * [dim_per_hidden_layer]

activation = torch.nn.ReLU
sigma_fun = 'softplus'  # Default is 'exp'
model = Autoencoder(x_dim=x_dim, z_dim=z_dim, hidden_layer_dims=hidden_layer_dims, raw_io=True,
                    x_stats=np.stack([x_train_mean, x_train_std]), z_stats=np.stack([z_train_mean, z_train_std]),
                    x_inv_masses=x_inv_masses, z_inv_masses=z_inv_masses,
                    stoch_enc=True, stoch_dec=True, activation=activation, sigma_fun=sigma_fun)

In [10]:
# Print model 
model

CondNoiseAutoencoder(
  (encoder): CondNoiseMLP(
    (sigma_fun): Softplus(beta=1, threshold=20)
    (output_nn): Sequential(
      (0): Linear(in_features=14, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=6, bias=True)
    )
    (cond_noise_nn): Sequential(
      (0): Linear(in_features=8, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=12, bias=True)
    )
  )
  (decoder): CondNoiseMLP(
    (sigma_fun): Softplus(beta=1, threshold=20)
    (output_nn): Sequential(
      (0): Linear(in_features=14, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=6, bias=True)
    )
    (cond_noise_nn): Sequential(
      (0): Linear(in_features=8, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=12, bias=True)
    )
  )
)

In [11]:
train_batch_size = 20000
train_loaders = DataLoader(dataset=x_train, batch_size=train_batch_size, shuffle=True), \
                DataLoader(dataset=z_train, batch_size=train_batch_size, shuffle=True)

## Loop through different hyperparameters and train/eval a model for each

In [20]:
verbose = True
save_dir = '.'

nus = [0, 10, 20, 50, 100, 200]
for nu in nus:
    print(f'Training with initial anchor loss coef nu = {nu}')

    # Reset seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    # Create a new model and optimizer
    model = Autoencoder(x_dim=x_dim, z_dim=z_dim, hidden_layer_dims=hidden_layer_dims, raw_io=True,
                    x_stats=np.stack([x_train_mean, x_train_std]), z_stats=np.stack([z_train_mean, z_train_std]),
                    x_inv_masses=x_inv_masses, z_inv_masses=z_inv_masses,
                    stoch_enc=True, stoch_dec=True, activation=activation, sigma_fun=sigma_fun)
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    
    #  Stage-1 training with the anchor penalty
    config['nu_e'] = config['nu_d'] = nu
    config['epochs'] = 400  # originally 200; increased to 400 to ensure convergence
    history = None
    eval_losses, history = train_and_val(model, train_loaders, eval_loaders, config, optimizer, verbose=verbose, 
                                          prev_hist=history, log_freq=100, lr_decay=False)
    # Stage-2 training without the anchor penalty
    config['nu_e'] = config['nu_d'] = 0
    config['epochs'] = 800  # originally 600; increased to 800 to ensure convergence
#     history = None
    eval_losses, history = train_and_val(model, train_loaders, eval_loaders, config, optimizer, verbose=verbose, 
                                          prev_hist=history, log_freq=100, lr_decay=False)
    # Save history in JSON-lines format
    ## convert pytorch float tensors into plain numpy float arrs in history
    for key, val in history.items():
        if isinstance(val, (list, np.ndarray)) :
            if not isinstance(np.sum(val), (int, np.integer)):  # my crude test to see if this is an array of float type
                history[key] = [float(n) for n in val]
    import pandas as pd
    df = pd.DataFrame(history)
    df.to_json(f'history-nu={nu}.jsonl', orient='records', lines=True)
    

    # Save trained model
    save_path = os.path.join(save_dir, f'swae-nu={nu}.pkl')
    torch.save(model.state_dict(), save_path)
    print('Saved model weights to', save_path)

    # Reset seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    # Collect model results
    model.to('cpu')
    model.encoder.output_stats.to('cpu')
    model.decoder.output_stats.to('cpu')

    all_arrs = {'train': {}, 'val': {}}  # This will store all numpy arrays of interest
    all_arrs['train']['x'] = x_train
    all_arrs['train']['z'] = z_train
    all_arrs['val']['x']   = x_val
    all_arrs['val']['z']   = z_val

    for data_key in 'train', 'val':
        arrs = all_arrs[data_key]
        arrs['z_decoded']       = model.decode(torch.from_numpy(arrs['z'])) # p_D(x) = \int_z p(z) p_D(x|z)  "x_pred_truth"
        arrs['x_encoded']       = model.encode(torch.from_numpy(arrs['x'])) # p_E(z) = \int_x p(x) p_E(z|x)  "z_pred"
        arrs['x_reconstructed'] = model.decode(arrs['x_encoded'])     # p_D(y) = \int_x \int_z p(x) p_E(z|x) p_D(y|z) "x_pred"

        # Feed the same z input to the decoder multiple times and study the stochastic of the output
        num_repeats = 100
        num_diff_zs = 100
        arrs['z_rep']         = np.array([np.repeat(arrs['z'][i:i+1], num_repeats, axis=0) for i in range(num_diff_zs)]) # "z_fixed"
        z_rep_tensor          = torch.from_numpy(arrs['z_rep'])                                                          # tmp
        arrs['z_decoded_rep'] = np.array([model.decode(z_rep_tensor[i]).detach().numpy() for i in range(num_diff_zs)])   # "x_pred_truth_fixed"
        arrs['x_rep']         = np.array([np.repeat(arrs['x'][i:i+1], num_repeats, axis=0) for i in range(num_diff_zs)]) # "x_fixed"

        # Convert all results to numpy arrays
        for (field, arr) in arrs.items():
            if isinstance(arr, torch.Tensor):
                arrs[field] = arr.detach().numpy()
    
    from plot_utils import plotFunction
    # Z-space
    data_key = 'val'
    arrs = all_arrs[data_key]
    dataList = [arrs['z'], arrs['x_encoded']]
    pltDim   = (2,4)
    numBins  = 50
    binsList = [np.linspace(-100.,100., numBins), 
                np.linspace(-100.,100., numBins), 
                np.linspace(-400.,400., numBins), 
                np.linspace(0.,400., numBins),
                np.linspace(-100.,100., numBins), 
                np.linspace(-100.,100., numBins), 
                np.linspace(-400.,400., numBins), 
                np.linspace(0.,400., numBins)]
    particleNameList = [r'$e^-$', r'$e^+$']

    fig = plotFunction(dataList = dataList, pltDim = pltDim, binsList = binsList, particleNameList = particleNameList, show=False)
    fig.savefig(os.path.join(save_dir, f'Z_marginals-nu={nu}.png'), bbox_inches='tight')
    plt.close(fig)
    
    # X-space
    # Set plotting parameters
    dataList = [arrs['x'], arrs['x_reconstructed'], arrs['z_decoded']]
    pltDim   = (2,4)
    numBins  = 50
    binsList = [np.linspace(-100.,100., numBins), 
                np.linspace(-100.,100., numBins), 
                np.linspace(-400.,400., numBins), 
                np.linspace(0.,400., numBins),
                np.linspace(-100.,100., numBins), 
                np.linspace(-100.,100., numBins), 
                np.linspace(-400.,400., numBins), 
                np.linspace(0.,400., numBins)]
    particleNameList = [r'$e^-$', r'$e^+$']

    fig = plotFunction(dataList = dataList, pltDim = pltDim, binsList = binsList, particleNameList = particleNameList, show=False)
    fig.savefig(os.path.join(save_dir, f'X_marginals-nu={nu}.png'), bbox_inches='tight')
    plt.close(fig)
    
    # Derived quantity (Z-boson mass)
    from func_utils import Zboson_mass
    ## Z-space
    dataList = [Zboson_mass(arrs['z']), Zboson_mass(arrs['x_encoded'])]
    pltDim   = (1,1)
    numBins  = 40
    binsList = [np.linspace(70.,110., numBins)]
    particleNameList = []
    nameList = [r'$M_Z$', r'Counts']
    fig = plotFunction(dataList = dataList, pltDim = pltDim, binsList=binsList, particleNameList=particleNameList, nameList=nameList, show=False)
    fig.savefig(os.path.join(save_dir, f'Z_derived-nu={nu}.png'), bbox_inches='tight')
    plt.close(fig)
    
    ## X-space
    dataList = [Zboson_mass(arrs['x']), Zboson_mass(arrs['x_reconstructed']), Zboson_mass(arrs['z_decoded'])] 
    pltDim   = (1,1)
    numBins  = 40
    binsList = [np.linspace(70.,110., numBins)]
    particleNameList = []
    nameList = [r'$M_Z$', r'Counts']

    # Create plot
    fig = plotFunction(dataList = dataList, pltDim = pltDim, binsList=binsList, particleNameList=particleNameList, nameList=nameList, show=False)
    fig.savefig(os.path.join(save_dir, f'X_derived-nu={nu}.png'), bbox_inches='tight')
    plt.close(fig)
    
    
    # Transport plots
    from plot_utils import fullTransportPlot 
    nzList    = [20,20,20,20,20,20,20,20]
    nxList    = [20,20,20,20,20,20,20,20]
    x_display_lims = [(-100, 100), (-100, 100), (-400, 400), (0, 400)]
    limzList = x_display_lims * 2
    limxList = x_display_lims * 2
    pltDim    = (2,4)
    titleList = [r'$p_x$',r'$p_y$',r'$p_z$',r'$E$','','','','']
    fig = fullTransportPlot(arrs['z'], arrs['z_decoded'][:, 0:8], nzList=nzList, nxList=nxList, limzList=limzList, limxList=limxList, pltDim=pltDim, titleList=titleList, show=False)
    fig.savefig(os.path.join(save_dir, f'z-z_decoded-transport-nu={nu}.png'), bbox_inches='tight')
    plt.close(fig)
    
    fig = fullTransportPlot(arrs['x_encoded'], arrs['x_reconstructed'][:, 0:8], nzList=nzList, nxList=nxList, limzList=limzList, limxList=limxList, pltDim=pltDim, titleList=titleList, show=False)
    fig.savefig(os.path.join(save_dir, f'x_encoded-x_reconstructed-transport-nu={nu}.png'), bbox_inches='tight')
    plt.close(fig)
    
    print()
    print()

Training with initial anchor loss coef nu = 0
{'num_hidden_layers': 1, 'dim_per_hidden_layer': 128, 'lr': 0.001, 'beta': 1.0, 'lamb': 1.0, 'tau': 0, 'rho': 0, 'nu_e': 0, 'nu_d': 0, 'epochs': 400}
epoch:	0
train -- loss:3736.88, x_loss:2947.43, z_loss:789.456, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:2336.41, x_loss:2917.46, z_loss:704.736, alt_x_loss:1631.68, anchor_loss:1.99303
epoch:	100
train -- loss:15.4132, x_loss:2.53477, z_loss:12.8784, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:10.0811, x_loss:2.21209, z_loss:2.76299, alt_x_loss:7.31814, anchor_loss:2.34349
epoch:	200
train -- loss:4.5008, x_loss:0.853854, z_loss:3.64694, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:6.11704, x_loss:1.21233, z_loss:2.2783, alt_x_loss:3.83874, anchor_loss:2.37631
epoch:	300
train -- loss:8.26902, x_loss:1.02083, z_loss:7.24818, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:4.60986, x_loss:1.67359, z_loss:1.97577, alt_x_loss:

eval -- loss:295.88, x_loss:2.40384, z_loss:4.85215, alt_x_loss:291.028, anchor_loss:0.288393
epoch:	100
train -- loss:8.20357, x_loss:2.22069, z_loss:5.98289, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:187.255, x_loss:2.42391, z_loss:3.67476, alt_x_loss:183.58, anchor_loss:0.316254
epoch:	200
train -- loss:4.35145, x_loss:1.32711, z_loss:3.02434, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:111.025, x_loss:1.26921, z_loss:2.82149, alt_x_loss:108.203, anchor_loss:0.301285
epoch:	300
train -- loss:5.36957, x_loss:1.12163, z_loss:4.24794, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:70.08, x_loss:1.15147, z_loss:3.13107, alt_x_loss:66.9489, anchor_loss:0.281126
epoch:	400
train -- loss:7.82801, x_loss:0.924961, z_loss:6.90305, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:48.3987, x_loss:0.642002, z_loss:3.45811, alt_x_loss:44.9406, anchor_loss:0.26366
epoch:	500
train -- loss:17.583, x_loss:0.818289, z_loss:16.7647, al

eval -- loss:26.0569, x_loss:0.530531, z_loss:1.74214, alt_x_loss:24.3148, anchor_loss:0.299962
epoch:	799
train -- loss:8.73799, x_loss:0.522676, z_loss:8.21531, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0
eval -- loss:24.7256, x_loss:0.970133, z_loss:3.42714, alt_x_loss:21.2985, anchor_loss:0.341218
Saved model weights to ./swae-nu=100.pkl


Training with initial anchor loss coef nu = 200
{'num_hidden_layers': 1, 'dim_per_hidden_layer': 128, 'lr': 0.001, 'beta': 1.0, 'lamb': 1.0, 'tau': 0, 'rho': 0, 'nu_e': 200, 'nu_d': 200, 'epochs': 400}
epoch:	0
train -- loss:4123.88, x_loss:2964.67, z_loss:792.926, alt_x_loss:0, x_constraint_loss:0, anchor_loss:1.83143
eval -- loss:2441.76, x_loss:2951.69, z_loss:720.754, alt_x_loss:1721.01, anchor_loss:1.7927
epoch:	100
train -- loss:26.1324, x_loss:9.82825, z_loss:14.9408, alt_x_loss:0, x_constraint_loss:0, anchor_loss:0.0068167
eval -- loss:125.682, x_loss:9.31919, z_loss:5.41894, alt_x_loss:120.263, anchor_loss:0.00995276
epoch:	200
trai